In [ ]:
import numpy as np 
import pandas as pd
import os

In [ ]:
!pip install --quiet ../input/albumentations051/albumentations-0.5.1-py3-none-any.whl

In [ ]:
tez_path = '../input/tez-lib/'
import sys
sys.path.append(tez_path)

In [ ]:
package_path = '../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0' #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
sys.path.append(package_path)

In [ ]:
import os
import albumentations
import pandas as pd

import tez
from tez.datasets import ImageDataset
from tez.callbacks import EarlyStopping

import torch
import torch.nn as nn
from torch.nn import functional as F

from efficientnet_pytorch import EfficientNet
from sklearn import metrics, model_selection, preprocessing

In [ ]:
class LeafModel(tez.Model):
    def __init__(self, num_classes):
        super().__init__()

        self.effnet = EfficientNet.from_name("efficientnet-b5")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(self.effnet._fc.in_features, num_classes)
        self.step_scheduler_after = "epoch"
        
    def monitor_metrics(self, outputs, targets):
        if targets is None:
            return {}
        outputs = torch.argmax(outputs, dim=1).cpu().detach().numpy()
        targets = targets.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, outputs)
        return {"accuracy": accuracy}
    
    def fetch_optimizer(self):
        opt = torch.optim.Adam(self.parameters(), lr=3e-4)
        return opt
    
    def fetch_scheduler(self):
        sch = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
            self.optimizer, T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1
        )
        return sch

    def forward(self, image, targets=None):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        
        if targets is not None:
            loss = nn.CrossEntropyLoss()(outputs, targets)
            metrics = self.monitor_metrics(outputs, targets)
            return outputs, loss, metrics
        return outputs, None, None

In [ ]:
model = LeafModel(num_classes=5)


In [ ]:
model.load("../input/myleafmodel/model-b5.bin")

In [ ]:
test_image_path = "../input/cassava-leaf-disease-classification/test_images/"
ids=[]
test_image_paths=[]
with os.scandir(test_image_path) as entries:
    for entry in entries:
        ids.append(entry.name)
        test_image_paths.append(os.path.join(test_image_path, entry.name))
test_image_paths


In [ ]:
targets=np.array([1]*len(test_image_paths))

In [ ]:
train_aug = albumentations.Compose([
            albumentations.RandomResizedCrop(256, 256),
            albumentations.Transpose(p=0.5),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.ShiftScaleRotate(p=0.5),
            albumentations.Normalize(
                mean=[0.485, 0.456, 0.406], 
                std=[0.229, 0.224, 0.225], 
                max_pixel_value=255.0, 
                p=1.0
            )], p=1.)
test_dataset = ImageDataset(
    image_paths=test_image_paths,
    targets=targets,
    resize=None,
    augmentations=train_aug,
)

In [ ]:
preds = model.predict(test_dataset, batch_size=32, n_jobs=-1, device="cuda")
final_preds = None
for p in preds:
    if final_preds is None:
        final_preds = p
    else:
        final_preds = np.vstack((final_preds, p))
final_preds = final_preds.argmax(axis=1)
df = pd.DataFrame(columns = ["image_id", "label"])
df.image_id=ids
df.label = final_preds
df.to_csv("submission.csv", index=False)